# Examples for AbsSystem Class (v1.3)

In [1]:
# suppress warnings for these examples
import warnings
warnings.filterwarnings('ignore')

# imports
import imp
from astropy.coordinates import SkyCoord
import astropy.units as u

from linetools.isgm import abssystem as lt_absys
from linetools.spectralline import AbsLine
from linetools.isgm.abscomponent import AbsComponent
from linetools.spectra import io

In [2]:
def data_path(filename):
    data_dir = '../../linetools/spectra/tests/files'
    return os.path.join(data_dir, filename)

## Simple instantiation

### Standard init

In [3]:
radec = SkyCoord(ra=123.1143*u.deg, dec=-12.4321*u.deg)
gensys = lt_absys.GenericAbsSystem(radec, 1.244, [-500,500]*u.km/u.s, NHI=16.)
gensys

<GenericAbsSystem: name=Foo type=Generic, 08:12:27.432 -12:25:55.56, z=1.244, NHI=16>

### From components

#### One component

In [4]:
# HI Lya, Lyb
lya = AbsLine(1215.670*u.AA, z=2.92939)
lya.limits.set([-300.,300.]*u.km/u.s)  # vlim
lyb = AbsLine(1025.7222*u.AA, z=lya.attrib['z'])
lyb.limits.set([-300.,300.]*u.km/u.s)  # vlim
abscomp = AbsComponent.from_abslines([lya,lyb])
abscomp.coord = radec

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [5]:
# HILyman system
HIsys = lt_absys.LymanAbsSystem.from_components([abscomp])
print(HIsys)
print(HIsys._components)

<LymanAbsSystem: name=J081227.432-122555.56_z2.929 type=HILyman, 08:12:27.432 -12:25:55.56, z=2.92939, NHI=0>
[<AbsComponent: 08:12:27.432 -12:25:55.56, Name=HI_z2.92939, Zion=(1,1), Ej=0 1 / cm, z=2.92939, vlim=-300 km / s,300 km / s>]


#### Multiple components

In [6]:
# SiII
SiIItrans = ['SiII 1260', 'SiII 1304', 'SiII 1526', 'SiII 1808']
abslines = []
for trans in SiIItrans:
    iline = AbsLine(trans, z=2.92939)
    iline.limits.set([-250.,80.]*u.km/u.s)  # vlim
    abslines.append(iline)
#
SiII_comp = AbsComponent.from_abslines(abslines)
SiII_comp.coord = radec

In [7]:
# Generic 
imp.reload(lt_absys)
LLSsys = lt_absys.GenericAbsSystem.from_components([abscomp,SiII_comp])
print(LLSsys)
print(LLSsys._components)

<GenericAbsSystem: name=Foo type=Generic, 08:12:27.432 -12:25:55.56, z=2.92939, NHI=0>
[<AbsComponent: 08:12:27.432 -12:25:55.56, Name=HI_z2.92939, Zion=(1,1), Ej=0 1 / cm, z=2.92939, vlim=-300 km / s,300 km / s>, <AbsComponent: 08:12:27.432 -12:25:55.56, Name=SiII_z2.92939, Zion=(14,2), Ej=0 1 / cm, z=2.92939, vlim=-250 km / s,80 km / s>]


## Methods

### List of AbsLines

In [8]:
lines = LLSsys.list_of_abslines()
lines

[<AbsLine: HI 1215, z=2.9294 wrest=1215.6700 Angstrom>,
 <AbsLine: HI 1025, z=2.9294 wrest=1025.7222 Angstrom>,
 <AbsLine: SiII 1260, z=2.9294 wrest=1260.4221 Angstrom>,
 <AbsLine: SiII 1304, z=2.9294 wrest=1304.3702 Angstrom>,
 <AbsLine: SiII 1526, z=2.9294 wrest=1526.7070 Angstrom>,
 <AbsLine: SiII 1808, z=2.9294 wrest=1808.0129 Angstrom>]

### Measure EW

In [9]:
spec = io.readspec(data_path('UM184_nF.fits'))
LLSsys.measure_restew(spec=spec)

### Measure AODM

In [10]:
LLSsys.measure_aodm(spec=spec)

You may now wish to update the component column densities with update_component_colm()


### Update Component Columns

In [11]:
LLSsys.update_component_colm()

### Single Line

In [12]:
lyb = LLSsys.get_absline('HI 1025')
lyb

<AbsLine: HI 1025, z=2.9294 wrest=1025.7222 Angstrom>

In [13]:
lyb = LLSsys.get_absline(1025.72*u.AA)
lyb

<AbsLine: HI 1025, z=2.9294 wrest=1025.7222 Angstrom>

In [14]:
lyb.wrest

<Quantity 1025.7222 Angstrom>

## Tables

### Table of Transitions from abslines

In [15]:
LLSsys.fill_trans()
LLSsys._trans

wrest,name,Z,ion,Ej,z,EW,sig_EW,flag_N,logN,sig_logN
Angstrom,,,,1 / cm,,Angstrom,Angstrom,,,
float64,str9,int64,int64,float64,float64,float64,float64,int64,float64,float64
1025.7222,HI 1025,1,1,0.0,2.92939,1.48584513486,0.0241451263249,2,15.6759375485,0.0508251192979
1215.67,HI 1215,1,1,0.0,2.92939,1.92726537784,0.0115671560839,2,14.9596058681,0.141161827446
1260.4221,SiII 1260,14,2,0.0,2.92939,0.431299149192,0.0129661024161,1,13.5883729709,0.0150745701489
1304.3702,SiII 1304,14,2,0.0,2.92939,0.0681045452711,0.0144099621049,1,13.7708705955,0.0862006463782
1526.707,SiII 1526,14,2,0.0,2.92939,0.111376644825,0.0168685426234,1,13.6707360009,0.0640855113383
1808.0129,SiII 1808,14,2,0.0,2.92939,-0.0195080735717,0.0210203404763,3,0.0,0.50976387151


### Table of Columns for Components

In [16]:
LLSsys.fill_ionN()
LLSsys._ionN

Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN
,,,1 / cm,,km / s,km / s,,,
int64,int64,int64,float64,float64,float64,float64,int64,float64,float64
1,1,0,0.0,2.92939,-300.0,300.0,2,15.6759375485,0.0508251192979
14,2,0,0.0,2.92939,-250.0,80.0,1,13.5944455609,0.014499668701
